In [38]:
import numpy as np
import torch


Training data
We can represent the training data using two matrices: inputs and targets, each with one row per observation, and one column per variable.

In [39]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73,67,423],
                  [91,88,64],
                  [87,134,58],
                  [102,43,37],
                   [69,96,70]], dtype='float32')
inputs

array([[ 73.,  67., 423.],
       [ 91.,  88.,  64.],
       [ 87., 134.,  58.],
       [102.,  43.,  37.],
       [ 69.,  96.,  70.]], dtype=float32)

In [40]:
#Targets(apples, organges)
targets =np.array([[56,70],
                  [81,101],
                  [119,133],
                  [22,37],
                  [103,119]], dtype='float32')

Let's convert the arrays to PyTorch tensors.


In [41]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets =torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67., 423.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Linear regression model from scratch
The weights and biases (w11, w12,... w23, b1 & b2) can also be represented as matrices, initialized as random values. The first row of w and the first element of b are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges.

In [42]:
# Weights and biases
w = torch.randn(2,3, requires_grad=True)
b = torch.randn(2,requires_grad=True)
print(w)
print(b)

tensor([[ 0.8926, -0.0213, -0.5826],
        [-0.6970,  0.0754, -0.6065]], requires_grad=True)
tensor([ 0.1550, -1.6677], requires_grad=True)


torch.randn creates a tensor with the given shape, with elements picked randomly from a normal distribution with mean 0 and standard deviation 1
Our model is simply a function that performs a matrix multiplication of the inputs and the weights w (transposed) and adds the bias b (replicated for each observation).
We can define the model as follows:

In [43]:
def model(x):
    return x @ w.t()+b

@ represents matrix multiplication in PyTorch, and the .t method returns the transpose of a tensor.
The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [44]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[-182.5574, -304.0278],
        [  42.2210,  -97.2726],
        [  41.1675,  -87.3781],
        [  68.7277,  -91.9575],
        [  18.9180,  -84.9745]], grad_fn=<AddBackward0>)


In [45]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


You can see a big difference between our model's predictions and the actual targets because we've initialized our model with random weights and biases. Obviously, we can't expect a randomly initialized model to just work.

Loss function
Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:
Calculate the difference between the two matrices (preds and targets).
Square all elements of the difference matrix to remove negative values.
Calculate the average of the elements in the resulting matrix.
The result is a single number, known as the mean squared error (MSE).

In [46]:
# MSE loss
def mse(t1,t2):
    diff = t1 - t2
    return torch.sum(diff*diff) / diff.numel()

torch.sum returns the sum of all the elements in a tensor. The .numel method of a tensor returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

In [47]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(35973.5547, grad_fn=<DivBackward0>)


Here’s how we can interpret the result: On average, each element in the prediction differs from the actual target by the square root of the loss. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. The result is called the loss because it indicates how bad the model is at predicting the target variables. It represents information loss in the model: the lower the loss, the better the model.

Compute gradients

In [48]:
# Compute gradients
loss.backward()

In [49]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[ 0.8926, -0.0213, -0.5826],
        [-0.6970,  0.0754, -0.6065]], requires_grad=True)
tensor([[ -5750.0889,  -7177.6069, -22412.5469],
        [-18349.5293, -19433.0508, -40546.9609]])


In [50]:
#Adjust weights and biases to reduce the loss
w
w.grad

tensor([[ -5750.0889,  -7177.6069, -22412.5469],
        [-18349.5293, -19433.0508, -40546.9609]])

In [51]:
with torch.no_grad():
    w -=w.grad*1e-5
    b -=b.grad*1e-5

In [52]:
# Let's verify that the loss is actually lower
loss = mse(preds, targets)
print(loss)

tensor(35973.5547, grad_fn=<DivBackward0>)


In [53]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


Train the model using gradient descent
Generate predictions
Calculate the loss
Compute gradients w.r.t the weights and biases
Adjust the weights by subtracting a small quantity proportional to the gradient
Reset the gradients to zero

#Train the model using gradient descent
Generate predictions
Calculate the loss
Compute gradients w.r.t the weights and biases
Adjust the weights by subtracting a small quantity proportional to the gradient
Reset the gradients to zero

In [75]:
# Generate predictions
preds1= model(inputs)
print(preds1)

tensor([[-2.1451e+01,  6.9357e-02],
        [ 8.1897e+01, -2.5148e+00],
        [ 8.3528e+01,  1.7156e+01],
        [ 9.4898e+01, -2.4918e+01],
        [ 5.9747e+01,  9.4578e+00]], grad_fn=<AddBackward0>)


In [76]:
# Calculate the loss
loss1 = mse(preds1, targets)
print(loss1)

tensor(5930.1382, grad_fn=<DivBackward0>)


In [77]:
# Compute gradients
loss1.backward()
print(w.grad)
print(b.grad)

tensor([[ -1682.9454,  -4352.5366, -14036.9688],
        [-15390.8828, -16998.4961, -21153.3848]])
tensor([-147.7955, -604.6745])


In [61]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [62]:
print(w)
print(b)

tensor([[ 0.9757,  0.0898, -0.2337],
        [-0.3980,  0.3944,  0.0103]], requires_grad=True)
tensor([ 0.1570, -1.6618], requires_grad=True)


In [ ]:
#to compare I will take the above w,b
tensor([[ 0.8926, -0.0213, -0.5826],
        [-0.6970,  0.0754, -0.6065]], requires_grad=True)
tensor([ 0.1550, -1.6677], requires_grad=True)

In [63]:
# Calculate loss
preds2 =model(inputs)
loss2=mse(preds2,targets)
print(loss2)

tensor(5930.1382, grad_fn=<DivBackward0>)


Train for multiple epochs
To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. Let's train the model for 100 epochs.

In [87]:
# Train for 100 epochs
for i in range(100):
    preds3=model(inputs)
    loss3 = mse(preds3, targets)
    loss3.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()
print(loss3)

tensor(80.4619, grad_fn=<DivBackward0>)


In [89]:
# Calculate loss
preds3 = model(inputs)
loss3 = mse(preds, targets)
print(loss3)

tensor(139.8800, grad_fn=<DivBackward0>)


In [90]:
# Predictions
preds3

tensor([[ 57.0749,  71.3761],
        [ 78.0055,  93.7075],
        [125.6436, 145.9825],
        [ 30.6398,  42.1734],
        [ 89.0421, 103.8400]], grad_fn=<AddBackward0>)

In [91]:
# Targets
targets


tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

The predictions are now quite close to the target variables. 

In [ ]:
jovian.commit(project='02-linear-regression')

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
